In [1]:
import math
import numpy as np
import torch
from higherOrderKME import sigkernel

In [2]:
seq_len = 5
x = np.arange(seq_len).reshape((1, seq_len, 1))
x

array([[[0],
        [1],
        [2],
        [3],
        [4]]])

In [33]:
# Specify the static kernel (for linear kernel use sigkernel.LinearKernel())
static_kernel = sigkernel.LinearKernel()

# Specify dyadic order for PDE solver (int > 0, default 0, the higher the more accurate but slower)
dyadic_order = 10
n_levels = 16

# Initialize the corresponding signature kernel
kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

In [34]:
# calculate signature levels
diff = (x[:,-1,:] - x[:,0,:]).squeeze()

x_levels = []
for i in range(n_levels+1):
    x_levels.append(diff**i / math.factorial(i))

x_levels

[1.0,
 4.0,
 8.0,
 10.666666666666666,
 10.666666666666666,
 8.533333333333333,
 5.688888888888889,
 3.250793650793651,
 1.6253968253968254,
 0.7223985890652558,
 0.28895943562610227,
 0.10507615840949175,
 0.03502538613649725,
 0.010777041888152998,
 0.003079154825186571,
 0.0008211079533830857,
 0.0]

In [35]:
np.dot(x_levels, x_levels)

427.56411567721267

In [36]:
xt = torch.tensor(x, dtype=torch.float64)
kernel.compute_kernel(xt, xt)

tensor([427.5643], dtype=torch.float64)

In [37]:
y = np.random.randint(10, size=seq_len).reshape((1, seq_len, 1))
y

array([[[4],
        [1],
        [8],
        [4],
        [7]]])

In [38]:
diff = (y[:,-1,:] - y[:,0,:]).squeeze()

y_levels = []
for i in range(n_levels+1):
    y_levels.append(diff**i / math.factorial(i))

y_levels

[1.0,
 3.0,
 4.5,
 4.5,
 3.375,
 2.025,
 1.0125,
 0.43392857142857144,
 0.1627232142857143,
 0.05424107142857143,
 0.016272321428571428,
 0.004437905844155844,
 0.001109476461038961,
 0.00025603302947052944,
 5.486422060082774e-05,
 1.0972844120165549e-05,
 2.0574082725310405e-06]

In [39]:
np.dot(y_levels, y_levels)

67.23440697647784

In [41]:
yt = torch.tensor(y, dtype=torch.float64)
kernel.compute_kernel(yt, yt)

tensor([67.2453], dtype=torch.float64)

In [42]:
np.dot(x_levels, y_levels)

157.75949587026497

In [43]:
kernel.compute_kernel(xt, yt)

tensor([157.7622], dtype=torch.float64)